In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt

import helper
import text_embeddings
import performance
import models
import predict
import constants

In [2]:
text_embedder = constants.TFIDF
text_embedder_args = {}
model_class = SGDClassifier
model_params = dict(eta0=0.0001,loss='hinge', random_state=15, penalty='l2', tol=1e-3, verbose=0)

In [3]:
svm = models.Model(model_class,
                model_params,
                text_embedder,
                text_embedder_args)

In [4]:
svm.initialize()

In [5]:
alpha_range = [ 0.0001,0.001,0.01,0.1,1,2,3,4,5,6,7,8,9,10,50,100]
params_search = {'alpha' : alpha_range}

svm.search_hyperparameter(params_search,
                         scoring='roc_auc')

,param_alpha,mean_train_score,mean_test_score,mean_train_score-mean_test_score
0,0.0001,0.984105,0.980899,0.003206
1,0.001,0.971835,0.970569,0.001265
2,0.01,0.958628,0.957825,0.000803
3,0.1,0.954920,0.954138,0.000782
4,1,0.954920,0.954138,0.000782
5,2,0.954920,0.954138,0.000782
6,3,0.954920,0.954138,0.000782
7,4,0.954920,0.954138,0.000782
8,5,0.954920,0.954138,0.000782
9,6,0.954920,0.954138,0.000782


In [18]:
svm.set_params(**dict(alpha=0.0001))
svm.fit()

In [ ]:
svm.model

In [19]:
svm.get_performance_measures()

{'accuracy': 0.9330470904457709,
 'f1': 0.9189177763686325,
 'confusion_matrix':              actual_0  actual_1
 predicted_0     31721      1183
 predicted_1      2653     21737}

In [20]:
svm.predict("I hate my life.")

{'class': 1}

# Test

In [ ]:
predict.manual_test(final_model, 
                         model_id="SVM",
                         vectorizer=vectorizer)

In [ ]:
test_results = predict.test_tweets_from_file(final_model,
                             "bot/tweets.json",
                             model_id="SVM",
                             vectorizer=vectorizer)

In [ ]:
print(test_results['is_depressed(model output)'].value_counts())

In [ ]:
depressed_tweets = test_results[test_results['is_depressed(model output)'] == 1]

In [ ]:
depressed_tweets.sort_values("model output probability (if any)", ascending=False).head(20)

# Future Work
- Explore string kernel https://github.com/timshenkao/StringKernelSVM
- word2vec using SVM (https://shop.tarjomeplus.com/UploadFileEn/TPLUS_EN_3959.pdf)
- https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/